# Bibliotecas

In [1]:
import pandas as pd
import geopy.distance as gp

# Carga dos Dados

In [185]:
poi = pd.read_csv('../data/base_pois_def.csv')
position = pd.read_csv('../data/posicoes.csv')

In [186]:
# Converte data_posicao para datetime
position['data_posicao'] = pd.to_datetime(position['data_posicao'], format="%a %b %d %Y %H:%M:%S GMT%z", exact=False)

# Determina se o veiculo estah parado (on nao) em cada registro
position['parado'] = position.apply(lambda x: (not x.ignicao and x.velocidade < 5), axis = 1)

# Calcula o tempo(s) entre dois registros consecutivos
position = position.sort_values(by=['placa','data_posicao'])
position['tempo'] = (position['data_posicao'] - position.groupby(['placa'])['data_posicao'].shift(1)).dt.seconds

# 'JOIN' dos dois DataFrames

In [188]:
data = []
for aPos in position.itertuples():
    elseWhere = True
    for aPoi in poi.itertuples():
        # Calcula a distancia do veiculo ao POI
        dist = gp.distance((aPos.latitude, aPos.longitude), (aPoi.latitude, aPoi.longitude)).meters
        # Cria um registro, se o veiculo estiver em um POI
        if (dist <= aPoi.raio):
            elseWhere = False
            data.append({'placa': aPos.placa, 'tempo': aPos.tempo, 'poi': aPoi.nome, 'parado': aPos.parado})

    # Cria um registro, se o veiculo estiver parado, fora de todos os POIs
    if (elseWhere and aPos.parado):
        data.append({'placa': aPos.placa, 'tempo': aPos.tempo, 'poi': '*', 'parado': aPos.parado})
        
positionPoi = pd.DataFrame(data)

# Consolidacao dos dados

In [196]:
result = positionPoi.groupby(['placa','poi','parado'], as_index=False)['tempo'].sum()
result

,placa,poi,parado,tempo
0,CAR0012,PONTO 24,False,1604.0
1,CAR0012,PONTO 24,True,342764.0
2,TESTE001,PONTO 1,False,3876.0
3,TESTE001,PONTO 1,True,471.0
4,TESTE001,PONTO 2,False,6001.0
5,TESTE001,PONTO 2,True,2432.0
6,TESTE001,PONTO 24,False,6408.0
7,TESTE001,PONTO 24,True,653235.0


In [208]:
# Quantidade de tempo que os veiculos passaram parados dentro de cada POI
result[result['parado']==True].groupby(['placa','poi'], as_index=False)['tempo'].sum()

,placa,poi,tempo
0,CAR0012,PONTO 24,342764.0
1,TESTE001,PONTO 1,471.0
2,TESTE001,PONTO 2,2432.0
3,TESTE001,PONTO 24,653235.0


In [204]:
# Quantidade de tempo que os veiculos estavam dentro de cada POI
result.groupby(['placa','poi'], as_index=False)['tempo'].sum()

,placa,poi,tempo
0,CAR0012,PONTO 24,344368.0
1,TESTE001,PONTO 1,4347.0
2,TESTE001,PONTO 2,8433.0
3,TESTE001,PONTO 24,659643.0


In [202]:
# Tempo total da frota gasto parado em cada POI
result[result['parado']==True].groupby(['poi'], as_index=False)['tempo'].sum()

,poi,tempo
0,PONTO 1,471.0
1,PONTO 2,2432.0
2,PONTO 24,995999.0


In [193]:
# Tempo total parado por veiculo, independente do POI
position[position['parado']==True].groupby(['placa'], as_index=False)['tempo'].sum()

,placa,tempo
0,CAR0012,342764.0
1,TESTE001,655750.0


In [214]:
poi.to_json()

'{"nome":{"0":"PONTO 1","1":"PONTO 2","2":"PONTO 3","3":"PONTO 4","4":"PONTO 5","5":"PONTO 6","6":"PONTO 7","7":"PONTO 8","8":"PONTO 9","9":"PONTO 10","10":"PONTO 11","11":"PONTO 12","12":"PONTO 13","13":"PONTO 14","14":"PONTO 15","15":"PONTO 16","16":"PONTO 17","17":"PONTO 18","18":"PONTO 19","19":"PONTO 20","20":"PONTO 21","21":"PONTO 22","22":"PONTO 23","23":"PONTO 24"},"raio":{"0":300,"1":300,"2":250,"3":250,"4":163,"5":170,"6":250,"7":250,"8":250,"9":250,"10":250,"11":250,"12":250,"13":250,"14":250,"15":250,"16":250,"17":250,"18":250,"19":250,"20":250,"21":250,"22":250,"23":350},"latitude":{"0":-25.5674270174,"1":-25.568056,"2":-25.414167,"3":-25.718611,"4":-25.3724045981,"5":-22.7182524062,"6":-25.336667,"7":-24.558056,"8":-24.940833,"9":-25.279167,"10":-25.498056,"11":-20.071389,"12":-21.678056,"13":-23.6525,"14":-8.112778,"15":-22.754444,"16":-25.419167,"17":-25.5375,"18":-25.152222,"19":-23.598889,"20":-22.124722,"21":-25.445278,"22":-25.547778,"23":-25.363333},"longitude":{"0